In [ ]:
#爬取搜狐体育信息
import requests
from bs4 import BeautifulSoup

#这里使用requests模块的get函数，因为我们从上可知网页的请求方式是：get#
res = requests.get('http://sports.sina.com.cn/')
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
#print(soup)
count = 0
link_set = list()
for news in soup.select('ul'):
    i = 0
    #print(news.select('li'))
    if len(news.select('a')) > 0:
        print(len(news.select('a')))
        length = len(news.select('a'))
        while(i < length):
            link2 = news.select('a')[i]['href']
            i = i + 1
            count = count + 1
            if(count > 62):
                if(link2.endswith(".shtml")and link2.startswith("https://")):
                    print(link2)
                    link_set.append(link2)
#print(link_set)
size = len(link_set)
print(size)

In [ ]:
from datetime import datetime
#写入数据库
import pymysql
pymysql.install_as_MySQLdb()
conn = pymysql.connect(host='localhost', user='root', passwd="newpassword", db='lucenedb2')
cur = conn.cursor()
def getNewsDetail(newsurl):
    result = {}
    res = requests.get(newsurl)
    res.encoding='utf-8'
    soup = BeautifulSoup(res.text,'html.parser')
   # print(soup)
    result['title'] = soup.select('.main-title')[0].text
    result['newssource'] = soup.select('.source')[0].text
    timeSource = soup.select('.date-source')[0].contents[1].text.strip()
    result['dt'] = datetime.strptime(timeSource, '%Y年%m月%d日 %H:%M')
   # print(soup.select('.article')[0].text[:180])
    result['article'] = soup.select('.article')[0].text
   #print(result)
    return result
flag = 0
for i in range(size):
    if(link_set[i] == "https://sports.sina.com.cn/sportsevents/3v3/2020-08-15/doc-iivhuipn8785301.shtml"):
        print("get here!")
        flag = 1
        i = i+1
    ret = getNewsDetail(link_set[i])
    sql_statement = "INSERT ignore INTO content(Title,content,link) VALUES(\'"+ret['title']+"\',\'"+ret['article'][:180]+"\',\'"+link_set[i]+"\')"
    if(flag == 1):
        print(sql_statement)
        try:
            cur.execute(sql_statement)
            conn.commit()
        except Exception as e:
            print(e)

In [ ]:
#爬取搜狐首页信息
import time
import requests
from bs4 import BeautifulSoup
import pymysql

def insert(title,summary,link):
    print("get here")
    db = pymysql.connect("localhost","root","newpassword","lucenedb2" ) 
    cursor = db.cursor()
    #summary = summary.tostring(summary,encoding='utf-8')
    title=pymysql.escape_string(title)
    summary=pymysql.escape_string(summary)
    sql="insert into content(Title,content,link) values('"+title+"','"+summary+"','"+link+"')"
    print(sql)
    cursor.execute(sql)
    db.commit()
    db.close()

res = requests.get('https://news.sina.com.cn/world/')#模拟get请求获取链接返回的内容
res.encoding = 'utf-8'#设置编码格式为utf-8
soup = BeautifulSoup(res.text, 'html.parser')#前面已经介绍将html文档格式化为一个树形结构，每个节点都是一个对python对象，方便获取节点内容
for new in soup.select('.news-item'):#BeautifulSoup提供的方法通过select选择想要的html节点类名，标签等，获取到的内容会被放到列表中
    if len(new.select('h2')) > 0:
        try:
            #加[0]是因为select获取内容后是放在list列表中[内容,],text可以获取标签中的内容
            info = requests.get(new.select('a')[0]['href'])
            info.encoding = 'utf-8'
            html = BeautifulSoup(info.text, 'html.parser')
            #print(html)
            title = html.select('.second-title')[0].text
            print (title)#获取大标题
            link = html.select('.source')[0]['href']
            print (link)
            article = []
            for v in html.select('.article p')[:-1]:#获取article中被p包含的内容去除最后一个p标签即责任编辑
                article.append(v.text.strip())#将内容添加到列表中，并去除两边特殊字符
            author_info = '\n'.join(article)#将列表中内容以换行连接
            print (author_info[:180])
            insert(title,author_info[:180],link)
        except:
            print("========无法处理=========")
      